In [15]:
import folium
import pandas as pd
import branca

In [ ]:
file_path = "./covid.xlsx"
df_covid = pd.read_excel(file_path, sheet_name="Sheet1")

In [9]:
covid_positivos = df_covid[
    (df_covid['RESDEFIN'].isin([
        'POSITIVO A SARS-COV-2', 
        'SARS-COV-2 - ALPHA', 
        'SARS-COV-2 - LAMBDA', 
        'SARS-COV-2 - DELTA'
    ])) |
    (df_covid['CLASCOVID19'].isin([
        'CONFIRMADO', 
        'CONF ASO', 
        'CONF DIC'
    ])) |
    (df_covid['ANTIGENCOVID'] == 'POSITIVO A SARS-COV-2')
]
covid_muertes = df_covid[
    (df_covid['EVOLUCI'] == 'DEFUNCION') | 
    (df_covid['FECDEF'].notna()) |
    (df_covid['DEFVERIFI'] == 'SI')  
]

positivos_x_estado = covid_positivos.groupby('ENTIDAD').size().reset_index(name='CASOS')
muertes_x_estado = covid_muertes.groupby('ENTIDAD').size().reset_index(name='DEFUNCIONES')
covid = positivos_x_estado.merge(muertes_x_estado, on='ENTIDAD', how='left').fillna(0)
covid

,ENTIDAD,CASOS,DEFUNCIONES
0,AGUASCALIENTES,2364,303
1,BAJA CALIFORNIA,5523,1257
2,BAJA CALIFORNIA SUR,2517,177
3,CAMPECHE,1131,192
4,CHIAPAS,1128,261
5,CHIHUAHUA,5361,1041
6,CIUDAD DE MEXICO,61964,5759
7,COAHUILA DE ZARAGOZA,7427,931
8,COLIMA,1182,183
9,DURANGO,3732,332


In [11]:
poblacion = {
    'AGUASCALIENTES': 1425607, 'BAJA CALIFORNIA': 3769020, 'BAJA CALIFORNIA SUR': 798447,
    'CAMPECHE': 928363, 'COAHUILA DE ZARAGOZA': 3146771, 'COLIMA': 731391, 'CHIAPAS': 5543828,
    'CHIHUAHUA': 3741869, 'CIUDAD DE MEXICO': 9209944, 'DURANGO': 1832650, 'GUANAJUATO': 6166934,
    'GUERRERO': 3540685, 'HIDALGO': 3082841, 'JALISCO': 8348151, 'MEXICO': 16992418,
    'MICHOACAN DE OCAMPO': 4748846, 'MORELOS': 1971520, 'NAYARIT': 1235456, 'NUEVO LEON': 5784442,
    'OAXACA': 4132148, 'PUEBLA': 6583278, 'QUERÉTARO': 2368467, 'QUINTANA ROO': 1857985,
    'SAN LUIS POTOSÍ': 2822255, 'SINALOA': 3026943, 'SONORA': 2944840, 'TABASCO': 2402598,
    'TAMAULIPAS': 3527735, 'TLAXCALA': 1342977, 'VERACRUZ DE IGNACIO DE LA LLAVE': 8062579,
    'YUCATAN': 2320898, 'ZACATECAS': 1622138
}

# Diccionario de coordenadas
estados_coords = {
    "AGUASCALIENTES": [21.8853, -102.2916],
    "BAJA CALIFORNIA": [32.6643, -115.4679],
    "BAJA CALIFORNIA SUR": [24.1426, -110.3129],
    "CAMPECHE": [19.8301, -90.5349],
    "COAHUILA DE ZARAGOZA": [25.4232, -101.0053],
    "COLIMA": [19.2433, -103.7246],
    "CHIAPAS": [16.7516, -93.1165],
    "CHIHUAHUA": [28.6329, -106.0691],
    "CIUDAD DE MEXICO": [19.4326, -99.1332],
    "DURANGO": [24.0277, -104.6537],
    "GUANAJUATO": [21.0190, -101.2574],
    "GUERRERO": [17.5734, -99.5056],
    "HIDALGO": [20.1108, -98.7395],
    "JALISCO": [20.6597, -103.3496],
    "MEXICO": [19.2833, -99.6536],
    "MICHOACAN DE OCAMPO": [19.7069, -101.1853],
    "MORELOS": [18.9211, -99.2307],
    "NAYARIT": [21.5039, -104.8950],
    "NUEVO LEON": [25.6866, -100.3161],
    "OAXACA": [17.0732, -96.7266],
    "PUEBLA": [19.0414, -98.2063],
    "QUERETARO": [20.5881, -100.3899],
     "QUINTANA ROO": [18.5141, -88.3038],
    "SAN LUIS POTOSI": [22.1565, -100.9855],
    "SINALOA": [24.8049, -107.3940],
    "SONORA": [29.0892, -110.9616],
    "TABASCO": [17.9895, -92.9281],
    "TAMAULIPAS": [23.7417, -99.1456],
    "TLAXCALA": [19.3133, -98.2394],
    "VERACRUZ DE IGNACIO DE LA LLAVE": [19.5433, -96.9147],
    "YUCATAN": [20.9674, -89.5926],
    "ZACATECAS": [22.7709, -102.5832]
}

covid['POBLACION'] = covid['ENTIDAD'].map(poblacion)
covid['LATITUD'] = covid['ENTIDAD'].apply(lambda x: estados_coords[x][0])
covid['LONGITUD'] = covid['ENTIDAD'].apply(lambda x: estados_coords[x][1])
covid.head()

,ENTIDAD,CASOS,DEFUNCIONES,POBLACION,LATITUD,LONGITUD
0,AGUASCALIENTES,2364,303,1425607.0,21.8853,-102.2916
1,BAJA CALIFORNIA,5523,1257,3769020.0,32.6643,-115.4679
2,BAJA CALIFORNIA SUR,2517,177,798447.0,24.1426,-110.3129
3,CAMPECHE,1131,192,928363.0,19.8301,-90.5349
4,CHIAPAS,1128,261,5543828.0,16.7516,-93.1165


In [14]:
covid['CASOS POR 100K'] = (covid['CASOS'] / covid['POBLACION']) * 100000
covid['DEFUNCIONES POR 100K'] = (covid['DEFUNCIONES'] / covid['POBLACION']) * 100000
covid.head()

,ENTIDAD,CASOS,DEFUNCIONES,POBLACION,LATITUD,LONGITUD,CASOS_POR_100K,DEFUNCIONES_POR_100K,CASOS POR 100K,DEFUNCIONES POR 100K
0,AGUASCALIENTES,2364,303,1425607.0,21.8853,-102.2916,165.824102,21.254104,165.824102,21.254104
1,BAJA CALIFORNIA,5523,1257,3769020.0,32.6643,-115.4679,146.536766,33.350845,146.536766,33.350845
2,BAJA CALIFORNIA SUR,2517,177,798447.0,24.1426,-110.3129,315.236954,22.168034,315.236954,22.168034
3,CAMPECHE,1131,192,928363.0,19.8301,-90.5349,121.827346,20.681565,121.827346,20.681565
4,CHIAPAS,1128,261,5543828.0,16.7516,-93.1165,20.346952,4.707938,20.346952,4.707938


In [16]:
colormap_cases = branca.colormap.linear.YlOrRd_09.scale(covid['CASOS'].min(), covid['CASOS'].max())
colormap_deaths = branca.colormap.linear.YlOrRd_09.scale(covid['DEFUNCIONES'].min(), covid['DEFUNCIONES'].max())
def add_title_legend(map_object, title):
    title_html = f"""
    <div style="position: fixed; 
                bottom: 120px; left: 30px; width: 220px; height: 60px;
                background-color: white; border:2px solid grey; z-index:9999;
                font-size:14px; font-weight: bold; padding: 10px;">
        <b>{title}</b>
    </div>
    """
    map_object.get_root().html.add_child(folium.Element(title_html))
def add_colorbar_legend(map_object, colormap):
    colorbar_html = f"""
    <div style="position: fixed; 
                bottom: 30px; left: 30px; width: 220px; height: 120px;
                background-color: white; border:2px solid grey; z-index:9999;
                font-size:14px; padding: 10px;">
        <b>Escala de colores</b><br>
        {colormap._repr_html_()}
    </div>
    """
    map_object.get_root().html.add_child(folium.Element(colorbar_html))

Inciso A

In [23]:
def create_map_cases_total():
    m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)
    
    for _, row in covid.iterrows():
        if pd.notna(row['CASOS']):
            color = colormap_cases(row['CASOS'])
            folium.CircleMarker(
                location=[row['LATITUD'], row['LONGITUD']], 
                radius=10,
                color=color,
                fill=True,
                fill_opacity=0.7,
                popup=f"{row['ENTIDAD']}: {row['CASOS']} casos"
            ).add_to(m)

    add_title_legend(m, "Casos de COVID-19")
    add_colorbar_legend(m, colormap_cases)
    return m

In [24]:
mapa_casos = create_map_cases_total()
mapa_casos.save('mapa_casos.html')

Inciso B

In [25]:
def create_map_deaths_total():
    m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)
    
    for _, row in covid.iterrows():
        if pd.notna(row['DEFUNCIONES']):
            color = colormap_deaths(row['DEFUNCIONES'])
            folium.CircleMarker(
                location=[row['LATITUD'], row['LONGITUD']], 
                radius=10,
                color=color,
                fill=True,
                fill_opacity=0.7,
                popup=f"{row['ENTIDAD']}: {row['DEFUNCIONES']} muertes"
            ).add_to(m)

    add_title_legend(m, "Muertes por COVID-19")
    add_colorbar_legend(m, colormap_deaths)
    return m

In [20]:
mapa_casos = create_map_cases_total()
mapa_casos.save('mapa_casos.html')

Inciso C. A

In [37]:
def create_map_cases_100k():
    m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)
    
    for _, row in covid.iterrows():
        if pd.notna(row['CASOS POR 100K']):
            color = colormap_cases(row['CASOS POR 100K'])
            folium.CircleMarker(
                location=[row['LATITUD'], row['LONGITUD']], 
                radius=10,
                color=color,
                fill=True,
                fill_opacity=0.7,
                popup=f"{row['ENTIDAD']}: {row['CASOS POR 100K']} casos por 100K"
            ).add_to(m)

    add_title_legend(m, "Casos por 100,000 Habitantes(Covid)")
    add_colorbar_legend(m, colormap_cases)
    return m

In [38]:
mapa_casos_100k = create_map_cases_100k()
mapa_casos_100k.save('mapa_casos_100k.html')

Inciso C. B

In [39]:
def create_map_deaths_100k():
    m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)
    
    for _, row in covid.iterrows():
        if pd.notna(row['DEFUNCIONES POR 100K']):
            color = colormap_deaths(row['DEFUNCIONES POR 100K'])
            folium.CircleMarker(
                location=[row['LATITUD'], row['LONGITUD']], 
                radius=10,
                color=color,
                fill=True,
                fill_opacity=0.7,
                popup=f"{row['ENTIDAD']}: {row['DEFUNCIONES POR 100K']} muertes por 100K"
            ).add_to(m)

    add_title_legend(m, "Muertes por 100,000 Habitantes(Covid)")
    add_colorbar_legend(m, colormap_deaths)
    return m

In [40]:
mapa_muertes_100k = create_map_deaths_100k()
mapa_muertes_100k.save('mapa_muertes_100k.html')